# About this notebook

This notebook will contain my manual analysis of some abundance files/metadata I am using for testing purposes. 

The idea is to get a "reference" of sort of what I would normally do, and compare this to what the LLM decides to do.

In [1]:
# Load libraries

library(tidyverse)
library(edgeR)
library(limma)
library(tximport)
library(DESeq2)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Loading required package: limma

Loading required package: S4Vectors

Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following object is masked from ‘package:limma’:

    plotMA


The following objects are masked from ‘package:lubridate’:

    intersect, setdiff, union


The following objects are masked from ‘package:dplyr’:

    combine, intersect, setdiff, union


The following objects are masked from ‘

In [40]:
# Load metadata

meta <- read_csv("~/work/notebooks//Testing/GSE268034/GSE268034_series_matrix_metadata.csv") %>%
mutate(genotype.clean = str_remove_all(genotype.ch1, " "))
meta

Rows: 12 Columns: 44
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (41): title, geo_accession, status, submission_date, last_update_date, t...
dbl  (3): channel_count, taxid_ch1, data_row_count

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


ERROR: [1m[33mError[39m in `mutate()`:[22m
[1m[22m[36mℹ[39m In argument: `genotype.clean = str_remove_all(meta$genotype.ch1, "
  ")`.
[1mCaused by error:[22m
[1m[22m[33m![39m `genotype.clean` must be size 12 or 1, not 0.


In [18]:
# List the abundance files. The tsv files should work...

files <- list.files(path = "~/work/data/kallisto_output/",
                    pattern = "abundance.tsv",
                    recursive = TRUE,
                    full.names = TRUE)
tx2gene <- read_tsv("~/work/data/kallisto_indices//human/t2g.txt",
                   col_names = FALSE) %>%
dplyr::select(1, 3) %>%
drop_na()
kallisto <- tximport(files = files,
         type = "kallisto",
         tx2gene = tx2gene,
         ignoreAfterBar = TRUE,
                    countsFromAbundance = "lengthScaledTPM")

Rows: 227665 Columns: 8
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr (6): X1, X2, X3, X4, X5, X8
dbl (2): X6, X7

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Note: importing `abundance.h5` is typically faster than `abundance.tsv`

reading in files with read_tsv

1 
2 
3 
4 
5 
6 
7 
8 
9 
10 
11 
12 


transcripts missing from tx2gene: 24683

summarizing abundance

summarizing counts

summarizing length



In [24]:
DGE <- DGEList(counts = kallisto$counts,
              samples = meta)

In [25]:
str(DGE)

Formal class 'DGEList' [package "edgeR"] with 1 slot
  ..@ .Data:List of 2
  .. ..$ : num [1:25668, 1:12] 0 0 0 0 0 0 0 0 0 0 ...
  .. .. ..- attr(*, "dimnames")=List of 2
  .. .. .. ..$ : chr [1:25668] "A1BG" "A1BG-AS1" "A1CF" "A2M" ...
  .. .. .. ..$ : chr [1:12] "Sample1" "Sample2" "Sample3" "Sample4" ...
  .. ..$ :'data.frame':	12 obs. of  47 variables:
  .. .. ..$ group                  : Factor w/ 1 level "1": 1 1 1 1 1 1 1 1 1 1 ...
  .. .. ..$ lib.size               : num [1:12] 26356 25952 28192 26604 25214 ...
  .. .. ..$ norm.factors           : num [1:12] 1 1 1 1 1 1 1 1 1 1 ...
  .. .. ..$ title                  : chr [1:12] "SUDHL4_LacZ_RGFP0_1" "SUDHL4_LacZ_RGFP0_2" "SUDHL4_LacZ_RGFP5_1" "SUDHL4_LacZ_RGFP5_2" ...
  .. .. ..$ geo_accession          : chr [1:12] "GSM8284502" "GSM8284503" "GSM8284504" "GSM8284505" ...
  .. .. ..$ status                 : chr [1:12] "Public on Aug 08 2024" "Public on Aug 08 2024" "Public on Aug 08 2024" "Public on Aug 08 2024" ...
  .. .. ..

I'm immediately noticing I will need to integrate the results.txt (or whatever) output from when I extracted the FASTQ files, since this is the way I will be using to link GEO accessions to the SRA IDs.

Just for testing purposes (to make sure all the command I intend can run on the container), I won't worry about this yet. However, it is something I will need to consider when building the framework.

For whatever reason this was VERY quick (quicker than when I run this on my laptop...)

In [29]:
keep.exprs <- filterByExpr(DGE,
                          group = DGE$samples$genotype.ch1)

In [30]:
DGE.filtered <- DGE[keep.exprs, keep.lib.sizes = FALSE]

In [32]:
dim(DGE)
dim(DGE.filtered)

[1] 25668    12

[1] 621  12

Ah. I imagine since I only am using a subset of the FASTQ files, that is why everything has been extraordinarily quick. 

In [33]:
DGE.final <- calcNormFactors(DGE.filtered)

In [36]:
design <- model.matrix(data = DGE.final$samples,
                       ~0 + genotype.ch1)
colnames(design) <- str_remove_all(colnames(design),
                                   "genotype.ch1")
contrast.matrix <- makeContrasts(
    KO = "GNAS knockout - WT",
    levels = colnames(design))

ERROR: Error in makeContrasts(KO = "GNAS knockout - WT", levels = colnames(design)): The levels must by syntactically valid names in R, see help(make.names).  Non-valid names: GNAS knockout


In [35]:
v <- voom(DGE.final,
          design)
vfit <- lmFit(v,
              design)
vfit <- contrasts.fit(vfit,
                      con

,GNAS knockout,WT
Sample1,0,1
Sample2,0,1
Sample3,0,1
Sample4,0,1
Sample5,1,0
Sample6,1,0
Sample7,1,0
Sample8,1,0
Sample9,1,0
Sample10,1,0
